In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Box dimensions
box_size = 10
num_circles = 100
initial_radius = box_size / 10  # r = 1/10 of the side length

# Generate random positions for 5 circles
circles = []
for _ in range(num_circles):
    x, y, z = np.random.uniform(initial_radius, box_size - initial_radius, 3)
    circles.append([x, y, z, initial_radius])

# Function to check if two circles touch or overlap
def circles_touch(c1, c2):
    x1, y1, z1, r1 = c1
    x2, y2, z2, r2 = c2
    distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
    return distance <= (r1 + r2)

# Grow circles until they touch another circle or the box walls
growing = [True] * num_circles  # Keep track of which circles are still growing
growth_rate = 0.1  # Rate at which radius increases

while any(growing):
    for i in range(num_circles):
        if not growing[i]:
            continue
        # Temporarily grow the circle
        circles[i][3] += growth_rate  

        x, y, z, r = circles[i]

        # Check if the circle touches the walls of the box
        if (x - r <= 0 or x + r >= box_size or
            y - r <= 0 or y + r >= box_size or
            z - r <= 0 or z + r >= box_size):
            growing[i] = False  # Stop growing this circle
            circles[i][3] -= growth_rate  # Undo last growth step
            continue

        # Check for collisions with other circles
        for j in range(num_circles):
            if i != j and circles_touch(circles[i], circles[j]):
                growing[i] = False  # Stop growing this circle
                circles[i][3] -= growth_rate  # Undo last growth step
                break

# Visualization
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': '3d'})

# Plot the circles
for x, y, z, r in circles:
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    xs = r * np.cos(u) * np.sin(v) + x
    ys = r * np.sin(u) * np.sin(v) + y
    zs = r * np.cos(v) + z
    ax.plot_wireframe(xs, ys, zs, color='b', alpha=0.6)  # Transparent blue spheres

# Draw the box (wireframe cube)
box_edges = [
    [[0, 0, 0], [box_size, 0, 0]], [[0, 0, 0], [0, box_size, 0]], [[0, 0, 0], [0, 0, box_size]],
    [[box_size, 0, 0], [box_size, box_size, 0]], [[box_size, 0, 0], [box_size, 0, box_size]],
    [[0, box_size, 0], [box_size, box_size, 0]], [[0, box_size, 0], [0, box_size, box_size]],
    [[0, 0, box_size], [box_size, 0, box_size]], [[0, 0, box_size], [0, box_size, box_size]],
    [[box_size, box_size, 0], [box_size, box_size, box_size]], [[box_size, 0, box_size], [box_size, box_size, box_size]],
    [[0, box_size, box_size], [box_size, box_size, box_size]],
]

for edge in box_edges:
    ax.plot3D(*zip(*edge), color="black")  # Black box outline

# Set limits
ax.set_xlim([0, box_size])
ax.set_ylim([0, box_size])
ax.set_zlim([0, box_size])

# Labels
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
ax.set_title("Spheres Growing Inside a 3D Box (Stopped at Walls)")

# Show the plot
plt.show()
